<a href="https://colab.research.google.com/github/suchartee/topic_modeling/blob/master/topic_modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# https://github.com/suchartee/topic_modeling

Libraries

In [0]:
import pandas as pd
import re
import requests
import string
import time
import sys
from bs4 import BeautifulSoup

# **Getting data**

In [0]:
# GET .JSON FILE FROM GITHUB
# read .json to list
with open('News_Category_Dataset_v2.json', 'r') as f:
    data = f.readlines()
# remove the trailing "\n" from each line
data = map(lambda x: x.strip(), data)
data_json_str = "[" + ','.join(data) + "]"
# load it into df
data_df = pd.read_json(data_json_str)

In [0]:
### THIS WOULD TAKE 3 DAYS TO GET ALL
# soup content from data_df['urls']
start = time.time()
t = []
c = []
for i, url in enumerate(data_df['link']):
    temp = re.sub(".*https?:\/\/", "http://", url)
    # get only huffpost website
    if 'huffingtonpost' in temp:
        try:
            headers = {'User-Agent': 'Chrome/50.0.2661.102'}
            html = requests.get(temp, headers=headers) 
            soup = BeautifulSoup(html.content, "html5lib")
            text = [p.text for p in soup.find_all("div", {"class": "content-list-component yr-content-list-text text"})]
            t.append(text)
            c.append(data_df['category'][i])
            print(i, temp, ((time.time() - start)/60)/60)
        except Exception:
            pass
print("Got all soup: " + str((time.time() - start)/60) + " mins")

In [0]:
# convert from df to xlsx for later uses
# join contents for each url
t = [" ".join(i) for i in t]
# df of transcript and category
df = pd.DataFrame([t, c]).transpose()
# name the columns
df.columns = ['transcript','category']
# remove all the blank content
df = df[df['transcript'] != '']
# export .xlsx
df.to_excel("transcript_category_no_index.xlsx", index=False)

# **Cleaning data**

In [0]:
# GET XLSX FILE FROM GITHUB
url = 'https://github.com/suchartee/topic_modeling/blob/master/data/transcript.xlsx?raw=true'
df = pd.read_excel(url)

In [0]:
def clean_text_huffpost(text):
  try:
    # lowercase
    text = text.lower()
    # remove punctuation
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    # remove special char
    text = re.sub('[‘’“”…―]|\n|\xa0|\w*\d\w*', ' ', text)
    # change extra whitespace to 1
    text = re.sub('[ ]{2,}', ' ', text).strip()
  except AttributeError:
    pass
  return text

In [19]:
df['transcript'] = df['transcript'].apply(lambda x: clean_text_huffpost(x))

nan


In [20]:
df.head()

,transcript,category
0,denton texas amanda painter sat at the kitchen...,CRIME
1,the fifa world cup starts june in russia and n...,ENTERTAINMENT
2,love actually turned to matrimony for hugh gra...,ENTERTAINMENT
3,rep adam schiff d calif and fellow democrats b...,ENTERTAINMENT
4,actress julianna margulies unleashed her funny...,ENTERTAINMENT


In [21]:
# remove stop words
!pwd

/content
